# Create Numpy Array
Jedna od varijacija metode za kreiranje podataka. Dohvaća slike s njihovih putanja, učitava ih kao array-e i appenda u array njihove klase kako bi se lakše spremale. Spremali smo svaku klasu zasebno, jer nastaju poteškoće oko spremanja datoteka koje su veće od 2GB. Ovdje smo 

In [ ]:

import torch
from tqdm import tqdm
import os
import json
import numpy as np
from random import shuffle
import cv2 
import sys

TRAINDATA = '/home/bosnmar/MachineLearning/Production/SelectiveLogo/'

DESTINATION = '/home/bosnmar/MachineLearning/Production/TrainingSet/FINAL_TRAINING/AUGDATA/'
IMGSIZE1 = 60
IMGSIZE2 = 40




def getLabels():
    with open('imgLabels.json') as f:
        return json.load(f)



def createTrainData(imglab, labels):
    trainingData  = []
    noOutputs = 0
    for lab in imglab:
        my_data = []
        saveFile = lab + "trainingData.npy"
        labPath1 = os.path.join(TRAINDATA, lab, "output")
        labPath2 = os.path.join(TRAINDATA, lab)
        if lab == "Other":
            pass
        else:
            print ("\nGoing throug class: {}; Size: {}\n".format(lab, len(os.listdir(labPath2 ))))

            if len(os.listdir(labPath2)) > 15000:
                print ("Klasa: {}\nImam više od 15k podataka, točnije {} podataka".format(lab, len(os.listdir(labPath2))))
                for img in tqdm(os.listdir(labPath2)):
                    if img == "output":
                        noOutputs = noOutputs + 1
                    else:
                        label = labels[lab]
                        cvimg = cv2.imread(os.path.join(labPath2, img), 0)
                        trImg = cv2.resize(cvimg, (IMGSIZE1, IMGSIZE2))
                        my_data.append([np.array(trImg), np.array(label)])

            
            else:
                print("Klasa: {};\nImam manje od 15k podataka".format(lab))
                i = len(os.listdir(labPath2))
                print("Imam {} podataka\n".format(i))
                for img in tqdm(os.listdir(labPath2)):
                    if img == "output":
                        noOutputs = noOutputs + 1
                    else:
                        label = labels[lab]
                        cvimg = cv2.imread(os.path.join(labPath2, img), 0)
                        trImg = cv2.resize(cvimg, (IMGSIZE1, IMGSIZE2))
                        my_data.append([np.array(trImg), np.array(label)])
                
                output=[]
                for img in tqdm(os.listdir(labPath1)):
                    if img == "output":
                        noOutputs = noOutputs + 1
                    else:
                        label = labels[lab]
                        cvimg = cv2.imread(os.path.join(labPath1, img), 0)
                        trImg = cv2.resize(cvimg, (IMGSIZE1, IMGSIZE2))
                        output.append([np.array(trImg), np.array(label)])
                shuffle(output)

                for elem in output:
                    my_data.append(elem)
                    i = i +1
                    if i > 15001:
                        break
                
            print("Imam ukupno {} elemenata".format(len(my_data)))
                
            np.save(DESTINATION + saveFile, my_data)
           
if __name__ == "__main__":

    labels = getLabels()
    imglab = [lab for lab in os.listdir(TRAINDATA)]

    createTrainData(imglab, labels)
